In [3]:
import cv2
import numpy as np

# YOLOv3 관련 파일 경로 설정
weights_path = 'yolov3/yolov3.weights'
config_path = 'yolov3/yolov3.cfg'
labels_path = 'yolov3/coco.names'

# YOLOv3 모델 초기화
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# 객체 탐지에 사용할 클래스 레이블 로딩
with open(labels_path, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# 영상 스트림 가져오기
stream_url = 'http://192.168.0.114:81/stream'  # 스트림 URL 설정
cap = cv2.VideoCapture(stream_url)

frame_count = 0  # 현재 프레임 수 초기화

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    frame_count += 1

    if frame_count % 15 != 0:  # 초당 1프레임만 처리 (30 프레임마다 처리)
        continue

    # 영상 프레임 크기 조정
    frame = cv2.resize(frame, None, fx=2, fy=2)  # 크기를 2배로 확대

    height, width, channels = frame.shape

    # YOLOv3 입력 이미지 전처리
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # 객체 탐지 결과 처리
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # 탐지 임계값 설정 (0.5 이상일 때만 탐지)
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # 객체의 경계 상자 좌표 계산
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                
                center_x = x + w / 2
                center_y = y + h / 2
                print(f"Object {i+1} center coordinates: ({center_x}, {center_y})")
                
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # 비최대 억제 수행 (같은 객체에 대한 중복 탐지 제거)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # 탐지된 객체를 화면에 표시
    font = cv2.FONT_HERSHEY_SIMPLEX
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = f'{classes[class_ids[i]]}: {confidences[i]:.2f}'
            color = (0, 255, 0)  # 객체 경계 상자의 색상 설정 (여기서는 녹색)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), font, 0.5, color, 2)

    # 화면에 영상 프레임 출력
    cv2.imshow('Object Detection', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 리소스 해제
cap.release()
cv2.destroyAllWindows()